# Medical Assistant Powered by the Azure OpenAI Service

In this sample we will explore how to use the Azure OpenAI Service to build a medical assistant. 

The doctor inputs Medical Shorthand (`shorthand`) and then we kick off the following stages:
1. Convert the Medical Shorthand to a Clinical Note. (`clinical_note`)
1. Convert the Medical Shorthand to a Structured Clinical Note. (`clinical_note_structured`) 
    1. Convert the Clinical Note to a Structured Clinical Note. (`clinical_note_structured`) - *variation*
1. Analyse the Clinical Note to check if the doctor has missed anything? (`feedback`)

## Azure OpenAI Service Setup

We will kick things off by importing the required libraries:

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.8"

#!import ../config/StringUtils.cs

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.8

Now, setting up the Azure OpenAI Service.

Note: You must to input your:
- Azure OpenAI Service Endpoint.
- Azure OpenAI Service Key.
- Azure OpenAI Service Model Deployment Name.

In [2]:
using Azure;
using Azure.AI.OpenAI;

var apiBase = "https://YOUR-ENDPOINT.openai.azure.com/"; // Add your endpoint here
var apiKey = "********************************";         // Add your api key here
var deploymentName = "text-davinci-003";                 // Add your deployment id here

OpenAIClient client = new OpenAIClient(
	new Uri(apiBase),
	new AzureKeyCredential(apiKey));

## Scenario Setup

Here we will setup the scenario by defining the Clinical Shorthand (`shorthand`) that we will be using for this sample.

Note: You can change the Clinical Shorthand to anything you like.

In [3]:
// This is a sample medical shorthand note written by a doctor after seeing a patient.
string shorthand = @"
John Brown, 73 M
Sx: CPOE and SOBOE 3/12 typical
PMHx: 
STEMI 2002. PCI LAD HREF.
CKD
CVA
AF on NOAC
MEDS bicor, aspirin, statin, rivaroxaban, entresto
OE
Well, BMI+ ECG NSR 80. BP 120/75
JVPNR. Nil pedal edema. HSDNM. Chest clear.
Imp
Stable. 
Plan
Lifestyle. Diet. GP f/u. PHT clinic because why not
";

## Process Medical Shorthand

Here we will use the Azure OpenAI Service to process the Medical Shorthand (`shorthand`).

### Step 1: Convert the Medical Shorthand to a Clinical Note

- **Input**: Medical Shorthand (`shorthand`)
- **Output**: Clinical Note (`clinicalNote`)
- **Model used**: `gpt-35-turbo`

> Notes: On trying the `gpt-4` model yeilded better results. You may need to tweak the API settings if/when you use a different model.

The template used is as follows:

```
<|im_start|>system
You are an medical assistant. 

Users will paste in medical shorthand. You will respond by converting this to a clinical note, expanding any abbreviations.
<|im_end|>
<|im_start|>user
******THE-MEDICAL-SHORTHAND-GOES-HERE******
<|im_end|>
<|im_start|>assistant
Clinical Note:
```

In [4]:
string promptStep1 = $@"
<|im_start|>system
You are an medical assistant. 

Users will paste in medical shorthand. You will respond by converting this to a clinical note, expanding any abbreviations.
<|im_end|>
<|im_start|>user
{shorthand}
<|im_end|>
<|im_start|>assistant
Clinical Note:
";

Response<Completions> completionsResponse1 = await client.GetCompletionsAsync(
    deploymentOrModelName: deploymentName,
    new CompletionsOptions()
    {
        Prompts = { promptStep1 },
        Temperature = (float)0.3,
        MaxTokens = 1000,
        FrequencyPenalty = (float)0,
        PresencePenalty = (float)0,
        StopSequences = {"<|im_end|>"},
        NucleusSamplingFactor = 1        
    });

Completions completions1 = completionsResponse1.Value;

string clinicalNote = completions1.Choices.First().Text;

Console.WriteLine($"Medical Shorthand: {shorthand}");
Console.WriteLine($"Clinical Note: {StringUtils.WordWrap(clinicalNote, 80)}");

Medical Shorthand: 
John Brown, 73 M
Sx: CPOE and SOBOE 3/12 typical
PMHx: 
STEMI 2002. PCI LAD HREF.
CKD
CVA
AF on NOAC
MEDS bicor, aspirin, statin, rivaroxaban, entresto
OE
Well, BMI+ ECG NSR 80. BP 120/75
JVPNR. Nil pedal edema. HSDNM. Chest clear.
Imp
Stable. 
Plan
Lifestyle. Diet. GP f/u. PHT clinic because why not

Clinical Note: John Brown, a 73 year old male, presented with chest pain of 12 days duration
typical of coronary artery disease. Past Medical History includes a ST-Elevation
Myocardial Infarction (STEMI) in 2002, which was treated with Percutaneous
Coronary Intervention (PCI) to the Left Anterior Descending Artery (LAD). He
also has Chronic Kidney Disease (CKD) and Cerebrovascular Accident (CVA). He is
on anticoagulation with a Non-Vitamin K Oral Anticoagulant (NOAC) for Atrial
Fibrillation (AF). His medications include Bicor, Aspirin, Statin, Rivaroxaban,
and Entresto. On examination, his Body Mass Index (BMI) was positive,
Electrocardiogram (ECG) showed Normal Sinus 

### Step 2 (A): Convert the Medical Shorthand to a Structured Clinical Note.

- **Input**: Medical Shorthand (`shorthand`)
- **Output**: Clinical Note (`clinicalNoteStructured`)
- **Model used**: `gpt-35-turbo`

> Notes: On trying the `gpt-4` model yeilded better results. You may need to tweak the API settings if/when you use a different model.

The template used is as follows:

```
<|im_start|>system
You are an medical assistant. 

Users will paste in medical shorthand. You will respond by converting this to a structured clinical note, expanding any abbreviations.
<|im_end|>
<|im_start|>user
******THE-MEDICAL-SHORTHAND-GOES-HERE******
<|im_end|>
<|im_start|>assistant
Clinical Note:
```

In [5]:
string promptStep2 = $@"
<|im_start|>system
You are an medical assistant. 

Users will paste in medical shorthand. You will respond by converting this to a structured clinical note, expanding any abbreviations.
<|im_end|>
<|im_start|>user
{shorthand}
<|im_end|>
<|im_start|>assistant
";

Response<Completions> completionsResponse2 = await client.GetCompletionsAsync(
    deploymentOrModelName: deploymentName,
    new CompletionsOptions()
    {
        Prompts = { promptStep2 },
        Temperature = (float)0.3,
        MaxTokens = 1000,
        FrequencyPenalty = (float)0,
        PresencePenalty = (float)0,
        StopSequences = {"<|im_end|>"},
        NucleusSamplingFactor = 1        
    });

Completions completions2 = completionsResponse2.Value;

string clinicalNoteStructured = completions2.Choices.First().Text;

Console.WriteLine($"Medical Shorthand: {shorthand}");
Console.WriteLine($"Structred Clinical Note: {StringUtils.WordWrap(clinicalNoteStructured, 80)}");

Medical Shorthand: 
John Brown, 73 M
Sx: CPOE and SOBOE 3/12 typical
PMHx: 
STEMI 2002. PCI LAD HREF.
CKD
CVA
AF on NOAC
MEDS bicor, aspirin, statin, rivaroxaban, entresto
OE
Well, BMI+ ECG NSR 80. BP 120/75
JVPNR. Nil pedal edema. HSDNM. Chest clear.
Imp
Stable. 
Plan
Lifestyle. Diet. GP f/u. PHT clinic because why not

Structred Clinical Note: John Brown is a 73 year old male with a history of ST Elevation Myocardial
Infarction in 2002, for which he underwent Percutaneous Coronary Intervention of
the Left Anterior Descending Artery. He also has Chronic Kidney Disease,
Cerebrovascular Accident, and is on a Non-Vitamin K Antagonist Oral
Anticoagulant (NOAC) for Atrial Fibrillation.

On examination, his Body Mass
Index is elevated, Electrocardiogram is normal sinus rhythm at 80 beats per
minute, Blood Pressure is 120/75, Jugular Venous Pressure is Normal, and there
is no pedal edema. His chest is clear.

The patient is stable. The plan is to
focus on lifestyle modifications, diet, follo

### Step 2 (B): Convert the Clinical Note to a Structured Clinical Note.

- **Input**: Medical Shorthand (`clinicalNote`)
- **Output**: Clinical Note (`clinicalNoteStructuredb`)
- **Model used**: `gpt-35-turbo`

> Notes: On trying the `gpt-4` model yeilded better results. You may need to tweak the API settings if/when you use a different model.

The template used is as follows:

```
<|im_start|>system
You are an medical assistant.

Users will paste in a clinical note. You will respond by converting this to a structured clinical note.
<|im_end|>
<|im_start|>user
******THE-LONG-FORM-CLINICAL-NOTE-GOES-HERE******
<|im_end|>
<|im_start|>assistant
Clinical Note:
```


In [7]:
string promptStep2b = $@"
<|im_start|>system
You are an medical assistant. 

Users will paste in a clinical note. You will respond by converting this to a structured clinical note.
<|im_end|>
<|im_start|>user
{clinicalNote}
<|im_end|>
<|im_start|>assistant
Clinical Note:
";

Response<Completions> completionsResponse2b = await client.GetCompletionsAsync(
    deploymentOrModelName: deploymentName,
    new CompletionsOptions()
    {
        Prompts = { promptStep2b },
        Temperature = (float)0.3,
        MaxTokens = 1000,
        FrequencyPenalty = (float)0,
        PresencePenalty = (float)0,
        StopSequences = {"<|im_end|>"},
        NucleusSamplingFactor = 1        
    });

Completions completions2b = completionsResponse2b.Value;

string clinicalNoteStructuredb = completions2b.Choices.First().Text;

Console.WriteLine($"Medical Shorthand: {shorthand}");
Console.WriteLine($"Structred Clinical Note: {StringUtils.WordWrap(clinicalNoteStructuredb, 80)}");

Medical Shorthand: 
John Brown, 73 M
Sx: CPOE and SOBOE 3/12 typical
PMHx: 
STEMI 2002. PCI LAD HREF.
CKD
CVA
AF on NOAC
MEDS bicor, aspirin, statin, rivaroxaban, entresto
OE
Well, BMI+ ECG NSR 80. BP 120/75
JVPNR. Nil pedal edema. HSDNM. Chest clear.
Imp
Stable. 
Plan
Lifestyle. Diet. GP f/u. PHT clinic because why not

Structred Clinical Note: John Brown is a 73 year old male who presented with chest pain of 12 days
duration, which is typical of coronary artery disease. His past medical history
includes a ST-Elevation Myocardial Infarction (STEMI) in 2002, which was treated
with Percutaneous Coronary Intervention (PCI) to the Left Anterior Descending
Artery (LAD). He also has Chronic Kidney Disease (CKD) and Cerebrovascular
Accident (CVA). He is on anticoagulation with a Non-Vitamin K Oral Anticoagulant
(NOAC) for Atrial Fibrillation (AF). His medications include Bicor, Aspirin,
Statin, Rivaroxaban, and Entresto. On examination, his Body Mass Index (BMI) was
positive, Electrocardiogr

### Step 3: Analyse the Clinical Note to check if the doctor has missed anything?

- **Input**: Medical Shorthand (`clinicalNote`)
- **Output**: Clinical Note (`feedback`)
- **Model used**: `gpt-35-turbo`

> Notes: On trying the `gpt-4` model yeilded better results. You may need to tweak the API settings if/when you use a different model.

The template used is as follows:

```
<|im_start|>system
You are an medical assistant.

Users will paste in a clinical note. You will respond by analysing the clinical note and checking if the doctor has missed anything.
<|im_end|>
<|im_start|>user
******THE-LONG-FORM-CLINICAL-NOTE-GOES-HERE******
<|im_end|>
<|im_start|>assistant
```


In [8]:
string promptStep3 = $@"
<|im_start|>system
You are an medical assistant. 

Users will paste in a clinical note. You will respond by analysing the clinical note and checking if the doctor has missed anything.
<|im_end|>
<|im_start|>user
{clinicalNote}
<|im_end|>
<|im_start|>assistant
Clinical Note:
";

Response<Completions> completionsResponse3 = await client.GetCompletionsAsync(
    deploymentOrModelName: deploymentName,
    new CompletionsOptions()
    {
        Prompts = { promptStep3 },
        Temperature = (float)0.3,
        MaxTokens = 1000,
        FrequencyPenalty = (float)0,
        PresencePenalty = (float)0,
        StopSequences = {"<|im_end|>"},
        NucleusSamplingFactor = 1        
    });

Completions completions3 = completionsResponse3.Value;

string feedback = completions3.Choices.First().Text;

Console.WriteLine($"Feedback: {StringUtils.WordWrap(feedback, 80)}");

Feedback: John Brown, a 73 year old male, presented with chest pain of 12 days duration
typical of coronary artery disease. Past Medical History includes a ST-Elevation
Myocardial Infarction (STEMI) in 2002, which was treated with Percutaneous
Coronary Intervention (PCI) to the Left Anterior Descending Artery (LAD). He
also has Chronic Kidney Disease (CKD) and Cerebrovascular Accident (CVA). He is
on anticoagulation with a Non-Vitamin K Oral Anticoagulant (NOAC) for Atrial
Fibrillation (AF). His medications include Bicor, Aspirin, Statin, Rivaroxaban,
and Entresto. On examination, his Body Mass Index (BMI) was positive,
Electrocardiogram (ECG) showed Normal Sinus Rhythm (NSR) at 80 beats per minute,
Blood Pressure (BP) was 120/75, Jugular Venous Pressure (JVP) was Normal, and
there was no Pedal Edema. His chest was clear.

Impression: Stable.

Plan:
Lifestyle modifications, diet, follow-up with General Practitioner (GP), and
referral to Pulmonary Hypertension (PHT) clinic.

Analysis:

